In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Configuration

In [2]:
MODEL_DIR = '/model'
DATA_FILE_NAME = 'traffic_data.csv'
CREATE_MODEL = False

# Import data from csv

In [3]:
df = pd.read_csv(DATA_FILE_NAME)
df['time'] = df['time'].astype('datetime64[ns]')
df.dtypes

time             datetime64[ns]
currentSpeed              int64
weatherDesc              object
motorcycles               int64
cars                      int64
heavyVehicles             int64
dtype: object

# Add new columns for further analysis

In [4]:
df['total_vehicle'] = df['cars'] + df['motorcycles'] + df['heavyVehicles']
df['Day'] = df['time'].dt.day_name()
df

,time,currentSpeed,weatherDesc,motorcycles,cars,heavyVehicles,total_vehicle,Day
0,2025-03-12 10:30:03,46,scattered clouds,0,13,0,13,Wednesday
1,2025-03-12 11:00:04,49,scattered clouds,1,9,0,10,Wednesday
2,2025-03-12 11:30:04,45,scattered clouds,1,13,0,14,Wednesday
3,2025-03-12 12:00:04,49,scattered clouds,0,5,0,5,Wednesday
4,2025-03-12 12:30:04,51,scattered clouds,4,11,0,15,Wednesday
...,...,...,...,...,...,...,...,...
846,2025-04-01 22:01:40,66,few clouds,0,0,0,0,Tuesday
847,2025-04-01 22:31:40,66,few clouds,0,1,0,1,Tuesday
848,2025-04-01 23:01:40,66,few clouds,0,0,0,0,Tuesday
849,2025-04-01 23:31:40,66,light rain,0,3,0,3,Tuesday


# EDA

In [5]:
df.describe()

,time,currentSpeed,motorcycles,cars,heavyVehicles,total_vehicle
count,851,851.000000,851.000000,851.000000,851.000000,851.000000
mean,2025-03-22 16:14:35.518213888,55.507638,0.846063,6.854289,0.245593,7.945946
min,2025-03-12 10:30:03,22.000000,0.000000,0.000000,0.000000,0.000000
25%,2025-03-17 02:45:23.500000,46.000000,0.000000,0.000000,0.000000,0.000000
50%,2025-03-23 02:30:13,66.000000,0.000000,5.000000,0.000000,6.000000
75%,2025-03-28 13:16:00,66.000000,1.000000,11.000000,0.000000,13.000000
max,2025-04-02 00:01:40,66.000000,13.000000,39.000000,3.000000,46.000000
std,NaN,11.760087,1.645255,7.639366,0.585784,8.836256


In [6]:
VEHICLE_FEATURE = ['cars', 'motorcycles',  'heavyVehicles']
FLOW_FEATURE = ['currentSpeed'] 

## Line chart showing relation between number of each type of vehicle and time

In [7]:
# Additional 30 min period column
base_time = pd.to_datetime('00:00:00')
total_minute = ((df['time'].dt.minute + (df['time'].dt.hour * 60)) // 30) * 30 
df['half_hour_interval'] = (base_time + pd.to_timedelta(total_minute, unit='m')).dt.time

In [8]:
px.line(df, x="time", y=VEHICLE_FEATURE)


## Line chart showing relation between number of each type of vehicle and time

In [9]:
px.line(df, x="time", y=FLOW_FEATURE)


## FLow and number of each vehicle on a specific date

In [10]:
def relation_with_time(selected_attr, title='title', xla='x', yla='y'):
    filtered_df = df.loc[df['time'].dt.date == pd.Timestamp('2025-03-30').date()]
    fig = px.line(filtered_df, x="time", y=selected_attr)
    fig.update_layout(
        title=title,
        xaxis_title=xla,
        yaxis_title=yla
    )
    fig.show()
    

relation_with_time(VEHICLE_FEATURE, 'Vehicle count on specific day', 'Time', 'Vehicle count')
relation_with_time(FLOW_FEATURE, 'Traffic flow on specific day', 'Time', 'Traffic flow')

df

,time,currentSpeed,weatherDesc,motorcycles,cars,heavyVehicles,total_vehicle,Day,half_hour_interval
0,2025-03-12 10:30:03,46,scattered clouds,0,13,0,13,Wednesday,10:30:00
1,2025-03-12 11:00:04,49,scattered clouds,1,9,0,10,Wednesday,11:00:00
2,2025-03-12 11:30:04,45,scattered clouds,1,13,0,14,Wednesday,11:30:00
3,2025-03-12 12:00:04,49,scattered clouds,0,5,0,5,Wednesday,12:00:00
4,2025-03-12 12:30:04,51,scattered clouds,4,11,0,15,Wednesday,12:30:00
...,...,...,...,...,...,...,...,...,...
846,2025-04-01 22:01:40,66,few clouds,0,0,0,0,Tuesday,22:00:00
847,2025-04-01 22:31:40,66,few clouds,0,1,0,1,Tuesday,22:30:00
848,2025-04-01 23:01:40,66,few clouds,0,0,0,0,Tuesday,23:00:00
849,2025-04-01 23:31:40,66,light rain,0,3,0,3,Tuesday,23:30:00


## Average number of each type of vehicle and traffic flow in each day of week

In [11]:
def avg_on_day_of_week(selected_attr, title='title', xla='x', yla='y'):
    
    color_map = {
        "Monday": "yellow",
        "Tuesday": "pink",
        "Wednesday": "green",
        "Thursday": "orange",
        "Friday": "blue",
        "Saturday": "purple",
        "Sunday": "red"
    }

    # Extract day names
    df['Day'] = df['time'].dt.day_name()

    grouped = df.groupby(['Day', 'half_hour_interval'])[selected_attr].mean().reset_index().sort_values('half_hour_interval', ascending=False)
    
    fig = px.line(
        grouped, 
        y=selected_attr, 
        x='half_hour_interval',
        color='Day',
        color_discrete_map=color_map,
    )
    fig.update_layout(
        xaxis=dict(title=xla, autorange="reversed"),
        title=title,
        yaxis_title=yla
    )
    fig.show()
    
avg_on_day_of_week(['total_vehicle'], 'Average vehicle count on each day of week every 30 minute', '30 minute interval', 'Vehicle count')
avg_on_day_of_week(FLOW_FEATURE, 'Average traffic flow on each day of week every 30 minute', '30 minute interval', 'traffic flow')




## Average number of each type of vehicle in each hour of day

In [12]:
def avg_on_hour(selected_attr, title='title', xla='x', yla='y'):
    byhour = df.groupby(df['half_hour_interval'])[selected_attr].mean()
    fig = px.line(byhour, y=selected_attr, x=byhour.index)
    fig.update_xaxes(
        tickmode='linear'
    )
    fig.update_layout(
        title=title,
        xaxis_title=xla,
        yaxis_title=yla
    )
    fig.show()

avg_on_hour(VEHICLE_FEATURE, 'Average vehicle type count every 30 minute of day', '30 minute interval', 'Vehicle count')
avg_on_hour(FLOW_FEATURE, 'Average traffic flow 30 minute of day', '30 minute interval', 'traffic flow')

## Average number of each vehicle type in each weather condition. 

In [13]:
def avg_on_weather_conditiopn(selected_attr, title='title', xla='x', yla='y'):
    notime = df.drop(columns=['time'])
    weather = notime.groupby('weatherDesc')[selected_attr].mean()

    fig = px.bar(weather, y=selected_attr, x=weather.index, barmode='group')
    fig.update_layout(
        title=title,
        xaxis_title=xla,
        yaxis_title=yla
    )
    fig.show()

avg_on_weather_conditiopn(VEHICLE_FEATURE, 'Average vehicle type count on each weather condition', 'weather condition', 'Vehicle count')
avg_on_weather_conditiopn(FLOW_FEATURE, 'Average traffic flow on each weather condition', 'weather condition', 'traffic flow')

## Convert hour to float

In [14]:
train_df = df.copy()

train_df['hour'] = (((train_df['time'].dt.hour * 60) + train_df['time'].dt.minute) // 30) / 2

train_df['weatherDesc'].unique()

array(['scattered clouds', 'few clouds', 'overcast clouds', 'light rain',
       'moderate rain', 'broken clouds', 'clear sky',
       'heavy intensity rain', 'haze', 'mist'], dtype=object)

## Convert day to is_weekend

In [15]:
def categorize_weather(wether_desc):
    
    if wether_desc in ['clear sky', 'few clouds', 'scattered clouds']:
        return 0
    
    if wether_desc in ['broken clouds', 'overcast clouds']:
        return 1
    
    if wether_desc in ['haze', 'mist']:
        return 2
    
    if wether_desc in ['light rain', 'moderate rain', 'heavy intensity rain']:
        return 3

train_df['ec_weather'] = train_df['weatherDesc'].map(categorize_weather)

train_df['is_weekend'] = train_df['Day'].map(
    lambda day: day in ['Saturday', 'Sunday']
).astype('int8')
train_df

,time,currentSpeed,weatherDesc,motorcycles,cars,heavyVehicles,total_vehicle,Day,half_hour_interval,hour,ec_weather,is_weekend
0,2025-03-12 10:30:03,46,scattered clouds,0,13,0,13,Wednesday,10:30:00,10.5,0,0
1,2025-03-12 11:00:04,49,scattered clouds,1,9,0,10,Wednesday,11:00:00,11.0,0,0
2,2025-03-12 11:30:04,45,scattered clouds,1,13,0,14,Wednesday,11:30:00,11.5,0,0
3,2025-03-12 12:00:04,49,scattered clouds,0,5,0,5,Wednesday,12:00:00,12.0,0,0
4,2025-03-12 12:30:04,51,scattered clouds,4,11,0,15,Wednesday,12:30:00,12.5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
846,2025-04-01 22:01:40,66,few clouds,0,0,0,0,Tuesday,22:00:00,22.0,0,0
847,2025-04-01 22:31:40,66,few clouds,0,1,0,1,Tuesday,22:30:00,22.5,0,0
848,2025-04-01 23:01:40,66,few clouds,0,0,0,0,Tuesday,23:00:00,23.0,0,0
849,2025-04-01 23:31:40,66,light rain,0,3,0,3,Tuesday,23:30:00,23.5,3,0


## Creating sequence function

In [16]:
def create_seq(data, seq_length):
    seq = []
    target = []
    for i in range(len(data) - seq_length):
        seq.append(data[i: i + seq_length, :-1])
        target.append(data[i: i + seq_length, -1])
    return torch.FloatTensor(seq), torch.FloatTensor(target)

## Create model class

In [17]:
class TrafficGRU(nn.Module):
    def __init__(self, weather_cat, hidden_size=64, num_layers=2):
        super().__init__()
        
        input_size = len(weather_cat) + 2 # Number of weather condition + sin/cos hour + is_weekend
        self.weather_cat_count = len(weather_cat)
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1) 
        self.ln = nn.LayerNorm(hidden_size) 
        self.relu = nn.ReLU(inplace=True)   

    def preprocess_param(self, hour, weather):

        sin_hour = torch.sin(2 * np.pi * hour / 24).unsqueeze(0)
        sin_cos = torch.cos(2 * np.pi * hour / 24).unsqueeze(0)
        
        int_weather = int(weather.item())
        weather_one_hot = torch.nn.functional.one_hot(
            torch.tensor(int_weather),
            num_classes=self.weather_cat_count
        ).float()
        
        return torch.cat([sin_cos, sin_hour, weather_one_hot])
        
    def forward(self, x):
        
        processed = []
        for i in x:
            
            batch = []
            
            for j in i:
                
                hour, weather = j[0], j[1]
                feature = self.preprocess_param(hour, weather)
                batch.append(feature)
            
            processed.append(torch.stack(batch))
        
        gru_input = torch.stack(processed)
        
        h0 = torch.zeros(self.gru.num_layers, x.size(0), self.gru.hidden_size)
        out, _ = self.gru(gru_input , h0)
        out = self.ln(out)
        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        return out

# Create and training model function

In [18]:
def create_and_train_model(target_feature, epoch, lr=0.001):
    weather_cat = list(train_df['ec_weather'].unique())

    # Set up hyper parameter
    hidden_size = 64
    num_layers = 2

    train_df.sort_values('time', inplace=True)
    train_set, car_test = np.split(train_df[['hour', 'ec_weather'] + [target_feature]], [int(len(train_df) * 0.8)])
    X, y = create_seq(train_set.values, 2)

    model = TrafficGRU(weather_cat, hidden_size, num_layers)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=24)

    for e in range(epoch): 
        for batch_x, batch_y in dataloader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {e+1}, Loss: {loss.item():.4f}')    

    X_test, y_test = create_seq(car_test.values, 2)
    test_criterion = nn.MSELoss()

    with torch.no_grad():
        predictions = model(X_test)
        test_loss = test_criterion(predictions, y_test)
    
    return  model, test_loss.item()    

# Train model of each target feature

In [27]:
car_model, car_per = create_and_train_model('cars', 50)
print("Number of car prediction model loss: ", car_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 68.4200
Epoch 2, Loss: 56.3572
Epoch 3, Loss: 45.9387
Epoch 4, Loss: 41.0721
Epoch 5, Loss: 38.9024
Epoch 6, Loss: 38.4073
Epoch 7, Loss: 38.2063
Epoch 8, Loss: 38.0800
Epoch 9, Loss: 37.8007
Epoch 10, Loss: 37.7052
Epoch 11, Loss: 37.8079
Epoch 12, Loss: 38.1701
Epoch 13, Loss: 38.6273
Epoch 14, Loss: 38.9277
Epoch 15, Loss: 39.1488
Epoch 16, Loss: 39.2724
Epoch 17, Loss: 39.3679
Epoch 18, Loss: 39.4320
Epoch 19, Loss: 39.4818
Epoch 20, Loss: 39.5214
Epoch 21, Loss: 39.5606
Epoch 22, Loss: 39.5963
Epoch 23, Loss: 39.6253
Epoch 24, Loss: 39.6458
Epoch 25, Loss: 39.6581
Epoch 26, Loss: 39.6632
Epoch 27, Loss: 39.6625
Epoch 28, Loss: 39.6576
Epoch 29, Loss: 39.6493
Epoch 30, Loss: 39.6397
Epoch 31, Loss: 39.6276
Epoch 32, Loss: 39.6137
Epoch 33, Loss: 39.6019
Epoch 34, Loss: 39.5891
Epoch 35, Loss: 39.5766
Epoch 36, Loss: 39.5643
Epoch 37, Loss: 39.5527
Epoch 38, Loss: 39.5424
Epoch 39, Loss: 39.5326
Epoch 40, Loss: 39.5233
Epoch 41, Loss: 39.5143
Epoch 42, Loss: 39.5060
E

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [20]:
mot_model, mot_per = create_and_train_model('motorcycles', 50)
print("Number of motorcycles prediction model loss: ", mot_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 10.4928
Epoch 2, Loss: 10.6142
Epoch 3, Loss: 10.7153
Epoch 4, Loss: 10.7452
Epoch 5, Loss: 10.6889
Epoch 6, Loss: 10.5865
Epoch 7, Loss: 10.4139
Epoch 8, Loss: 10.1796
Epoch 9, Loss: 9.9534
Epoch 10, Loss: 9.7052
Epoch 11, Loss: 9.6475
Epoch 12, Loss: 9.1293
Epoch 13, Loss: 9.2276
Epoch 14, Loss: 8.7409
Epoch 15, Loss: 8.5914
Epoch 16, Loss: 8.7068
Epoch 17, Loss: 8.4377
Epoch 18, Loss: 8.4178
Epoch 19, Loss: 8.5177
Epoch 20, Loss: 8.4414
Epoch 21, Loss: 8.5217
Epoch 22, Loss: 8.4881
Epoch 23, Loss: 8.4447
Epoch 24, Loss: 8.4713
Epoch 25, Loss: 8.4905
Epoch 26, Loss: 8.4806
Epoch 27, Loss: 8.4744
Epoch 28, Loss: 8.4817
Epoch 29, Loss: 8.4817
Epoch 30, Loss: 8.4779
Epoch 31, Loss: 8.4750
Epoch 32, Loss: 8.4727
Epoch 33, Loss: 8.4687
Epoch 34, Loss: 8.4642
Epoch 35, Loss: 8.4589
Epoch 36, Loss: 8.4536
Epoch 37, Loss: 8.4472
Epoch 38, Loss: 8.4409
Epoch 39, Loss: 8.4335
Epoch 40, Loss: 8.4265
Epoch 41, Loss: 8.4179
Epoch 42, Loss: 8.4106
Epoch 43, Loss: 8.4008
Epoch 44, Lo

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [21]:
heavy_model, heavy_per = create_and_train_model('heavyVehicles', 50)
print("Number of heavy vehicles prediction model loss: ", heavy_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 0.2500
Epoch 2, Loss: 0.2500
Epoch 3, Loss: 0.2500
Epoch 4, Loss: 0.2500
Epoch 5, Loss: 0.2500
Epoch 6, Loss: 0.2500
Epoch 7, Loss: 0.2500
Epoch 8, Loss: 0.2500
Epoch 9, Loss: 0.2500
Epoch 10, Loss: 0.2500
Epoch 11, Loss: 0.2500
Epoch 12, Loss: 0.2500
Epoch 13, Loss: 0.2500
Epoch 14, Loss: 0.2500
Epoch 15, Loss: 0.2500
Epoch 16, Loss: 0.2500
Epoch 17, Loss: 0.2500
Epoch 18, Loss: 0.2500
Epoch 19, Loss: 0.2500
Epoch 20, Loss: 0.2500
Epoch 21, Loss: 0.2500
Epoch 22, Loss: 0.2500
Epoch 23, Loss: 0.2500
Epoch 24, Loss: 0.2500
Epoch 25, Loss: 0.2500
Epoch 26, Loss: 0.2500
Epoch 27, Loss: 0.2500
Epoch 28, Loss: 0.2500
Epoch 29, Loss: 0.2500
Epoch 30, Loss: 0.2500
Epoch 31, Loss: 0.2500
Epoch 32, Loss: 0.2500
Epoch 33, Loss: 0.2500
Epoch 34, Loss: 0.2500
Epoch 35, Loss: 0.2500
Epoch 36, Loss: 0.2500
Epoch 37, Loss: 0.2500
Epoch 38, Loss: 0.2500
Epoch 39, Loss: 0.2500
Epoch 40, Loss: 0.2500
Epoch 41, Loss: 0.2500
Epoch 42, Loss: 0.2500
Epoch 43, Loss: 0.2500
Epoch 44, Loss: 0.25

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [22]:
speed_model, speed_per = create_and_train_model('currentSpeed', 200, 0.01)
print("Traffic flow prediction model loss: ", speed_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 730.1993
Epoch 2, Loss: 171.3672
Epoch 3, Loss: 111.6419
Epoch 4, Loss: 93.3165
Epoch 5, Loss: 81.0421
Epoch 6, Loss: 92.3761
Epoch 7, Loss: 89.9926
Epoch 8, Loss: 87.3914
Epoch 9, Loss: 83.5195
Epoch 10, Loss: 87.9922
Epoch 11, Loss: 98.2055
Epoch 12, Loss: 97.4846
Epoch 13, Loss: 85.6456
Epoch 14, Loss: 89.3439
Epoch 15, Loss: 90.5493
Epoch 16, Loss: 89.7251
Epoch 17, Loss: 90.0464
Epoch 18, Loss: 89.6532
Epoch 19, Loss: 89.4056
Epoch 20, Loss: 88.2301
Epoch 21, Loss: 87.2471
Epoch 22, Loss: 86.5642
Epoch 23, Loss: 86.0895
Epoch 24, Loss: 85.1414
Epoch 25, Loss: 83.9579
Epoch 26, Loss: 83.0524
Epoch 27, Loss: 81.7749
Epoch 28, Loss: 80.6040
Epoch 29, Loss: 80.1205
Epoch 30, Loss: 78.6952
Epoch 31, Loss: 77.3844
Epoch 32, Loss: 76.3198
Epoch 33, Loss: 75.4287
Epoch 34, Loss: 74.5323
Epoch 35, Loss: 73.4649
Epoch 36, Loss: 72.5916
Epoch 37, Loss: 72.2567
Epoch 38, Loss: 71.6126
Epoch 39, Loss: 71.3483
Epoch 40, Loss: 71.3337
Epoch 41, Loss: 70.2327
Epoch 42, Loss: 70.476

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [23]:
total_model, total_per = create_and_train_model('total_vehicle', 100)
print("Total number of vehicles prediction model loss: ", total_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([6, 2])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 137.0069
Epoch 2, Loss: 121.2238
Epoch 3, Loss: 105.2679
Epoch 4, Loss: 95.7161
Epoch 5, Loss: 90.0010
Epoch 6, Loss: 87.5105
Epoch 7, Loss: 86.1740
Epoch 8, Loss: 85.5276
Epoch 9, Loss: 85.1692
Epoch 10, Loss: 84.9180
Epoch 11, Loss: 84.8659
Epoch 12, Loss: 84.3392
Epoch 13, Loss: 84.4004
Epoch 14, Loss: 84.6171
Epoch 15, Loss: 85.1426
Epoch 16, Loss: 85.5928
Epoch 17, Loss: 85.9701
Epoch 18, Loss: 86.2746
Epoch 19, Loss: 86.4911
Epoch 20, Loss: 86.6393
Epoch 21, Loss: 86.7357
Epoch 22, Loss: 86.7909
Epoch 23, Loss: 86.8158
Epoch 24, Loss: 86.8169
Epoch 25, Loss: 86.8000
Epoch 26, Loss: 86.7727
Epoch 27, Loss: 86.7267
Epoch 28, Loss: 86.6724
Epoch 29, Loss: 86.6128
Epoch 30, Loss: 86.5469
Epoch 31, Loss: 86.4819
Epoch 32, Loss: 86.4175
Epoch 33, Loss: 86.3536
Epoch 34, Loss: 86.2911
Epoch 35, Loss: 86.2283
Epoch 36, Loss: 86.1687
Epoch 37, Loss: 86.1102
Epoch 38, Loss: 86.0544
Epoch 39, Loss: 86.0022
Epoch 40, Loss: 85.9511
Epoch 41, Loss: 85.9027
Epoch 42, Loss: 85.854

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([169, 2])) that is different to the input size (torch.Size([169, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



# Prediction model performance (MSE)

In [28]:
from math import sqrt

print(f"""
MSE of:
Number of car prediction model: {car_per:.4}
Number of motorcycle prediction model: {mot_per:.4}
Number of heavy vehicle prediction model: {heavy_per:.4}
Total number vehicle prediction model: {total_per:.4}
Traffic flow prediction model: {speed_per:.4}
""")


MSE of:
Number of car prediction model: 14.05
Number of motorcycle prediction model: 2.184
Number of heavy vehicle prediction model: 0.497
Total number vehicle prediction model: 17.55
Traffic flow prediction model: 34.62



In [25]:
import os

def export_model(model, model_name): 
    model.eval()
    scripted_model = torch.jit.script(model)  # or torch.jit.trace(model, input_example)
    
    if (not os.path.isdir('model')):
        os.mkdir('model')
    scripted_model.save(f"{MODEL_DIR}{model_name}.pt")

if (CREATE_MODEL):
    export_model(car_model, "car_num_model")
    export_model(mot_model, "motorcycle_vehicle_num_model")
    export_model(heavy_model, "heavy_vehicle_num_model")
    export_model(total_model, "total_vehicle_num_model")
    export_model(speed_model, "traffic_flow_model")